<a href="https://colab.research.google.com/github/abinjoabraham/DCFNet_pytorch/blob/master/TCN_on_Sequential_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from sample_data.utils import data_generator
from sample_data.model import TCN
import numpy as np
import argparse
from tensorboardX import SummaryWriter




In [0]:
torch.manual_seed(1111)
if torch.cuda.is_available():
   if not True:
      print("WARNING: You have a CUDA device, so you should probably run with --cuda")

root = './_sample_data/mnist'
batch_size = 64
n_classes = 10
input_channels = 1
seq_length = int(784 / input_channels)
epochs = 2
steps = 0
permuter = False
seed = 1111
nhid = 25
optimi = 'Adam'
lr = 2e-3
loginterval = 100
levels = 8
cuda = True
clip = -1 

In [3]:
!pip install tensorboardX


     |████████████████████████████████| 133kB 2.9MB/s 


In [0]:
train_loader, test_loader = data_generator(root, batch_size)
permute = torch.Tensor(np.random.permutation(784).astype(np.float64)).long()
channel_sizes = [25] * 8
kernel_size = 7
model = TCN(input_channels, n_classes, channel_sizes, kernel_size=kernel_size, dropout=0.05)





In [0]:
if torch.cuda.is_available():
  model.cuda()
  permute = permute.cuda()

lr = 2e-3
optimizer = getattr(optim, optimi)(model.parameters(), lr=lr)

In [0]:
def train(ep):
    global steps
    train_loss = 0
    model.train()
    #writer = SummaryWriter()

    for batch_idx, (data, target) in enumerate(train_loader):

        if cuda: data, target = data.cuda(), target.cuda()
        data = data.view(-1, input_channels, seq_length)
        if permuter:
            data = data[:, :, permute]
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        #writer.add_graph(model, data, True)             # model, input
        output = model(data)
        loss = F.nll_loss(output, target)
        #writer.add_scalar('lossfunction', loss, batch_idx)
        loss.backward()
        # writer.add_image('data/scalar1', data, batch_idx)
        # writer.add_image('data/scalar2', target, batch_idx)
        if clip > 0:
            torch.nn.utils.clip_grad_norm(model.parameters(), args.clip)
        optimizer.step()
        train_loss += loss
        steps += seq_length
        if batch_idx > 0 and batch_idx % loginterval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tSteps: {}'.format(
                ep, batch_idx * batch_size, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), train_loss.item()/loginterval, steps))
            train_loss = 0


In [0]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data = data.view(-1, input_channels, seq_length)
        if permuter:
            data = data[:, :, permute]
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss


In [7]:
    for epoch in range(1, epochs+1):
        train(epoch)
        test()
        if epoch % 10 == 0:
            lr /= 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.135190	Steps: 79184
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.467878	Steps: 157584
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.321196	Steps: 235984
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.260094	Steps: 314384
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.235185	Steps: 392784
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.216135	Steps: 471184
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.212920	Steps: 549584
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.204784	Steps: 627984
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.184365	Steps: 706384


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1504, Accuracy: 9509/10000 (95%)

Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.143648	Steps: 814576
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.158009	Steps: 892976
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.145578	Steps: 971376
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.135736	Steps: 1049776
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.135413	Steps: 1128176
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.131401	Steps: 1206576
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.132465	Steps: 1284976
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.139712	Steps: 1363376
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.114955	Steps: 1441776

Test set: Average loss: 0.0918, Accuracy: 9696/10000 (96%)

